In [1]:
# !pip install nibabel nilearn scikit-learn pandas numpy matplotlib
import os
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt

from nilearn.maskers import NiftiMasker
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
from scipy.stats import pearsonr

%matplotlib inline

## Load PsychPY timing data

In [12]:
# Convert start/stop times to scan indices
def time_to_index(times, TR=1.5):
    return np.round(np.array(times) / TR).astype(int)

In [ ]:
def get_index_range(start_times, stop_times, TR=1.5):
    indices = []
    for start, stop in zip(start_times, stop_times):
        start_idx, stop_idx = time_to_index([start, stop], TR)
        indices.extend(range(start_idx, stop_idx))
    return sorted(set(indices))
    
def convert_psychpy_time_to_fmri_index(beh_file):
    psychopy_df = pd.read_csv(beh_file)
    # Extract and clean start/stop columns
    view_start = psychopy_df['view.started'].dropna().values
    view_stop = psychopy_df['view.stopped'].dropna().values

    recall_start = psychopy_df['recall.started'].dropna().values
    recall_stop = psychopy_df['recall.stopped'].dropna().values

    imagine_start = psychopy_df['imagine_condition.started'].dropna().values
    imagine_stop = psychopy_df['imagine_condition.stopped'].dropna().values

    view_indices = get_index_range(view_start, view_stop)
    recall_indices = get_index_range(recall_start, recall_stop)
    imagine_indices = get_index_range(imagine_start, imagine_stop)
    
    return view_indices, recall_indices, imagine_indices

In [18]:
# Example usage for subject 1:
subj_behavior_file = {1:"psychopy_data/1_fmri design_2025-03-04_15h31.40.655.csv",
                      3:"psychopy_data/2_fmri design_2025-03-05_15h36.18.659.csv",
                      4:"psychopy_data/4_fmri design_2025-03-05_14h28.37.417.csv"
                      }
index = convert_psychpy_time_to_fmri_index(subj_behavior_file[1])

[15, 16, 17, 18, 19, 20, 32, 33, 34, 35, 36, 37, 38, 49, 50, 51, 52, 53, 54, 55, 67, 68, 69, 70, 71, 72, 99, 100, 101, 102, 103, 104, 105, 117, 118, 119, 120, 121, 122, 134, 135, 136, 137, 138, 139, 140, 151, 152, 153, 154, 155, 156, 157, 184, 185, 186, 187, 188, 189, 190, 201, 202, 203, 204, 205, 206, 207, 219, 220, 221, 222, 223, 224, 236, 237, 238, 239, 240, 241, 242, 269, 270, 271, 272, 273, 274, 286, 287, 288, 289, 290, 291, 292, 303, 304, 305, 306, 307, 308, 309, 321, 322, 323, 324, 325, 326, 353, 354, 355, 356, 357, 358, 359, 371, 372, 373, 374, 375, 376, 388, 389, 390, 391, 392, 393, 394, 405, 406, 407, 408, 409, 410, 411, 438, 439, 440, 441, 442, 443, 455, 456, 457, 458, 459, 460, 461, 472, 473, 474, 475, 476, 477, 478, 490, 491, 492, 493, 494, 495]


## Load the fMRI data

In [35]:
# Define data paths
data_dir = "/jukebox/hasson/snastase/neu502b-2025/neu502b-fmri/data/bids/derivatives/fmriprep" 
subjects = ["sub-01/func/", "sub-03/func/", "sub-04/func/"]  # Assuming 20 subjects, update as needed
task_prefix = "imagine"

In [8]:
output_dir = "neural_activity"
os.makedirs(output_dir, exist_ok=True)

def extract_neural_activity(subject):
    print(f"Processing {subject}...")

    # Find the preprocessed BOLD image
    bold_file = None
    for f in os.listdir(os.path.join(data_dir, subject)):
        if task_prefix in f and "desc-preproc_bold.nii.gz" in f:
            bold_file = os.path.join(data_dir, subject, f)
            break

    if not bold_file:
        print(f"No preprocessed BOLD file found for {subject}.")
        return

    # Load fMRI data
    bold_img = nib.load(bold_file)

    # Find brain mask
    mask_file = None
    for f in os.listdir(os.path.join(data_dir, subject)):
        if task_prefix in f and "desc-brain_mask.nii.gz" in f:
            mask_file = os.path.join(data_dir, subject, f)
            break

    if mask_file:
        mask_img = nib.load(mask_file)
        print(f"Using brain mask: {mask_file}")
    else:
        print(f"No explicit brain mask found. Will compute one.")
        mask_img = None
    
    confound_tsv = None
    for f in os.listdir(os.path.join(data_dir, subject)):
        if (task_prefix in f) and ("desc-confounds_timeseries.tsv" in f):
            confound_tsv = os.path.join(data_dir, f)
            break

    confound_data = None
    if confound_tsv is not None:
        df_conf = pd.read_csv(confound_tsv, sep='\t')
        # pick some columns, e.g., 6 motion parameters
        nuisance_cols = ['trans_x','trans_y','trans_z','rot_x','rot_y','rot_z']
        nuisance_cols = [c for c in nuisance_cols if c in df_conf.columns]
        confound_data = df_conf[nuisance_cols].fillna(method='bfill').fillna(method='ffill').values

# ---------
    
    # Extract time series from masked brain voxels
    masker = NiftiMasker(mask_img=mask_img, 
                         standardize=True,
                        high_pass=0.01)
    masker.fit(bold_img)
    time_series = masker.transform(bold_img, confounds=confound_data)
    
    print(f"Extracted neural activity shape (TRs x voxels): {time_series.shape}")  # (T, N_voxels)
    return time_series

In [37]:
# Run processing for all subjects
for subject in subjects:
    extract_neural_activity(subject)

Processing sub-01/func/...
Using brain mask: /jukebox/hasson/snastase/neu502b-2025/neu502b-fmri/data/bids/derivatives/fmriprep/sub-01/func/sub-01_task-imagine_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz


/usr/people/yx7967/miniconda3/envs/myenv/lib/python3.8/site-packages/nilearn/maskers/nifti_masker.py:110: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(


Extracted neural activity shape: (535, 73291)
Processing sub-03/func/...
Using brain mask: /jukebox/hasson/snastase/neu502b-2025/neu502b-fmri/data/bids/derivatives/fmriprep/sub-03/func/sub-03_task-imagine_space-T1w_desc-brain_mask.nii.gz
Extracted neural activity shape: (535, 60764)
Processing sub-04/func/...
Using brain mask: /jukebox/hasson/snastase/neu502b-2025/neu502b-fmri/data/bids/derivatives/fmriprep/sub-04/func/sub-04_task-imagine_space-T1w_desc-brain_mask.nii.gz
Extracted neural activity shape: (535, 56978)


## Multivariate pattern analysis (whole brain analysis)
* Train the binary category classifier on the view condition
* Test it on the recall and imagine condition (also cross validation)

See notebook: fmri-4/fmri-4-mvpa-key.ipynb

Ref handbook: https://brainhack-princeton.github.io/handbook/content_pages/05-02-mvpa.html


## Representational Similarity Analysis (RSA) 

* Keep the category the same (e.g. dog), what is the similarity between view, recall, and imagine conditions
* Compare the within-category similarity (within dogs vs within flowers)
* Expect flower category to be more clustered because of visual similarity
* Produce correlation matrix

See notebook: fmri-5/fmri-5-rsa-key.ipynb
